In [1]:
from fit.datamodules.tomo_rec import MNISTTomoFourierTargetDataModule
from fit.utils import convert2FC, fft_interpolate
from fit.utils.tomo_utils import get_proj_coords_pol, get_img_coords_pol

from fit.modules import TRecTransformerModule

from matplotlib import pyplot as plt

import torch

import numpy as np

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
seed_everything(22122020)

22122020

In [3]:
dm = MNISTTomoFourierTargetDataModule(root_dir='/data/mnist/', batch_size=32, num_angles=15)
dm.setup()

In [4]:
det_len = dm.gt_ds.get_ray_trafo().geometry.detector.shape[0]

In [5]:
proj_r, proj_phi, src_flatten = get_proj_coords_pol(angles=dm.gt_ds.get_ray_trafo().geometry.angles, 
                                                          det_len=det_len)
target_r, target_phi, dst_flatten, order = get_img_coords_pol(img_shape=dm.IMG_SHAPE, det_len=det_len)

In [7]:
model = TRecTransformerModule(d_model=256, y_coords_proj=proj_r, x_coords_proj=proj_phi,
                             y_coords_img=target_r, x_coords_img=target_phi, 
                             src_flatten_coords=src_flatten, dst_flatten_coords=dst_flatten, 
                             dst_order=order,
                             angles=dm.gt_ds.get_ray_trafo().geometry.angles, img_shape=dm.IMG_SHAPE,
                             detector_len=det_len,
                             loss='prod', use_fbp=True,
                             init_bin_factor=1, bin_factor_cd=5,
                             lr=0.0001, weight_decay=0.01, attention_type='linear', n_layers=4,
                             n_heads=8, d_query=256//8, dropout=0.1, attention_dropout=0.1,
                             encoder_only=True)

In [8]:
trainer = Trainer(max_epochs=20, 
                  gpus=1,
                  checkpoint_callback=ModelCheckpoint(
                                            filepath=None,
                                            save_top_k=1,
                                            verbose=False,
                                            save_last=True,
                                            monitor='Train/avg_val_mse',
                                            mode='min',
                                            prefix='best_val_loss_'
                                        ), 
                  deterministic=True)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model, datamodule=dm);


  | Name | Type        | Params
-------------------------------------
0 | trec | TRecEncoder | 3.2 M 
-------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/tbuchhol/.local/lib/python3.7/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/tbuchhol/Gitrepos/FourierImageTransformer/fit/utils/RAdam.py:119: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


In [12]:
model = TRecTransformerModule.load_from_checkpoint('lightning_logs/version_11/checkpoints/best_val_loss_-last.ckpt', 
                           y_coords_proj=model.y_coords_proj,
                           x_coords_proj=model.x_coords_proj,
                           y_coords_img=model.y_coords_img,
                           x_coords_img=model.x_coords_img,
                           angles=model.angles,
                           src_flatten_coords=model.src_flatten_coords,
                           dst_flatten_coords=model.dst_flatten_coords,
                           dst_order=model.dst_order)

In [13]:
test_res = trainer.test(model, datamodule=dm)

/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


bin_factor set to 1.

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Mean PSNR': array(27.06388282775879, dtype=float32),
 'SEM PSNR': array(0.019822554662823677, dtype=float32),
 'Train/avg_bin_mse': tensor(0.0074, device='cuda:0'),
 'Train/avg_val_amp_loss': tensor(13.1782, device='cuda:0'),
 'Train/avg_val_loss': tensor(19.0442, device='cuda:0'),
 'Train/avg_val_mse': tensor(0.0295, device='cuda:0'),
 'Train/avg_val_phi_loss': tensor(1.4367, device='cuda:0'),
 'bin_mse': tensor(0.0075, device='cuda:0'),
 'val_loss': tensor(19.0353, device='cuda:0'),
 'val_mse': tensor(0.0295, device='cuda:0')}
--------------------------------------------------------------------------------


In [19]:
model = TRecTransformerModule.load_from_checkpoint('lightning_logs/version_8/checkpoints/best_val_loss_-epoch=99-step=170181.ckpt', 
                           y_coords_proj=model.y_coords_proj,
                           x_coords_proj=model.x_coords_proj,
                           y_coords_img=model.y_coords_img,
                           x_coords_img=model.x_coords_img,
                           angles=model.angles,
                           src_flatten_coords=model.src_flatten_coords,
                           dst_flatten_coords=model.dst_flatten_coords,
                           dst_order=model.dst_order)

In [20]:
test_res_best = trainer.test(model, datamodule=dm)

/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


bin_factor set to 1.

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Mean PSNR': array(26.002174377441406, dtype=float32),
 'SEM PSNR': array(0.018735822290182114, dtype=float32),
 'Train/avg_bin_mse': tensor(0.0074, device='cuda:0'),
 'Train/avg_val_amp_loss': tensor(6.7612, device='cuda:0'),
 'Train/avg_val_loss': tensor(8.9109, device='cuda:0'),
 'Train/avg_val_mse': tensor(0.0393, device='cuda:0'),
 'Train/avg_val_phi_loss': tensor(1.2357, device='cuda:0'),
 'bin_mse': tensor(0.0075, device='cuda:0'),
 'val_loss': tensor(8.9044, device='cuda:0'),
 'val_mse': tensor(0.0393, device='cuda:0')}
--------------------------------------------------------------------------------
